# Tire kicker

Use this notebook to play with the code. The primary goal is to reproduce [Levi Wolf's Guinness delivery example](https://gist.github.com/ljwolf/e5927ab8c859ed477f496329c1ce19fc#file-guinness-py). 

In [1]:
import geopandas as gpd
import pandas, numpy, pyvrp, sys

In [2]:
sys.path.insert(0, '/home/dylan/projects/gsoc2025/spopt/') # not the published spopt

In [3]:
import spopt
print(spopt.__file__)

/home/dylan/projects/gsoc2025/spopt/spopt/__init__.py


In [4]:
from spopt.route import engine, heuristic, utils

## Reproduce the guinness example

In [5]:
from spopt.route.heuristic import LastMile
from pyvrp import stop

In [6]:
trucks = pandas.DataFrame(
    [['big', 'lng',      2000,    280, .004,  .50, 5],
     ['big', 'electric', 2000,    480, .002,  .50, 5],
     ['med', 'lng',      800, 280*.66, .0001, .63, 10],
     ['med', 'electric', 800, 480*.66, .004,  .50, 10],
     ['smo', 'lng',      400, 280*0.4, .002,  .50, 20],
     ['smo', 'electric', 400, 480*0.4, .0001, .63, 20],
     ],
     columns = [
         'namesize', 'namefuel', 'capacity', 
         'fixed_cost', 'cost_per_meter', 'cost_per_minute', 'n_truck'
         ]
)

In [7]:
dublin_pubs = gpd.read_file('/home/dylan/projects/gsoc2025/spopt/notebooks/gsoc2025/data/dublinpubs.geojson')

In [8]:
dublin_pubs.shape

(551, 8)

In [9]:
gdf = dublin_pubs

In [10]:
depot = gdf.iloc[0,:]
clients = gdf.iloc[1:,:].reset_index(drop=True)
clients = clients.set_index(clients.osmid.astype(str))

In [11]:
print('initializing model')
m = LastMile(
    depot_location=(depot.longitude.item(), depot.latitude.item()),
    depot_open=pandas.Timestamp("2030-01-02 07:00:00"),
    depot_close=pandas.Timestamp("2030-01-02 20:00:00"),
    depot_name=depot['name'],
)
print("adding clients")

m.add_clients(
    locations = clients.geometry, 
    delivery = clients.demand,
    pickup = clients.supply,
    time_windows=None,
    service_times=(numpy.log(clients.demand)**2).astype(int)
)
print("adding trucks")
m.add_trucks_from_frame(
    trucks, 
)

initializing model
adding clients
adding trucks


In [12]:
m.depot_location

(-6.28688, 53.341972)

In [13]:
m.solve(stop=pyvrp.stop.MaxRuntime(300))

/home/dylan/projects/gsoc2025/spopt/spopt/route/engine.py:31: UserWarning: Failed to connect to routing engine... using haversine distance and (d/500)**.75 for durations
  warnings.warn(


PyVRP v0.11.2

Solving an instance with:
    1 depot
    550 clients
    70 vehicles (6 vehicle types)

                  |       Feasible        |      Infeasible
    Iters    Time |   #      Avg     Best |   #      Avg     Best
H     500    171s |  38 42414290 35156325 |  49 38225972 36488027

Search terminated in 300.15s after 987 iterations.
Best-found solution has cost 35027521.

Solution results
    # routes: 12
     # trips: 12
   # clients: 550
   objective: 35027521
    distance: 181508
    duration: 1114
# iterations: 987
    run-time: 300.15 seconds



/home/dylan/projects/gsoc2025/spopt/spopt/route/engine.py:72: UserWarning: Failed to connect to routing engine... constructed routes will be straight lines and may not follow the road network.
  warnings.warn(
/home/dylan/projects/gsoc2025/spopt/spopt/route/engine.py:72: UserWarning: Failed to connect to routing engine... constructed routes will be straight lines and may not follow the road network.
  warnings.warn(
/home/dylan/projects/gsoc2025/spopt/spopt/route/engine.py:72: UserWarning: Failed to connect to routing engine... constructed routes will be straight lines and may not follow the road network.
  warnings.warn(
/home/dylan/projects/gsoc2025/spopt/spopt/route/engine.py:72: UserWarning: Failed to connect to routing engine... constructed routes will be straight lines and may not follow the road network.
  warnings.warn(
/home/dylan/projects/gsoc2025/spopt/spopt/route/engine.py:72: UserWarning: Failed to connect to routing engine... constructed routes will be straight lines and 

In [14]:
dir(m)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_setup_graph',
 'add_clients',
 'add_truck_type',
 'add_trucks_from_frame',
 'clients_',
 'cost_unit',
 'demand_unit_',
 'depot_',
 'depot_close',
 'depot_location',
 'depot_name',
 'depot_open',
 'explore',
 'model',
 'result_',
 'routes_',
 'solve',
 'stops_',
 'trucks_',
 'write_result']

A convention is that the underscore after the parameter, e.g. `value_`, is a solved value, not an input for the model

In [15]:
m.write_result("tire-kicker")

In [16]:
m.stops_

,route_name,stop_idx,eta,stop_number,target_uid,delivery,pickup,service_time,open_1,close_1,geometry
520,arctic-charbonneau,533,2030-01-02 07:02:00,1,1026167146,16,7,10,2030-01-02 07:00:00,2030-01-02 20:00:00,POINT (-6.28684 53.35424)
521,arctic-charbonneau,311,2030-01-02 07:04:00,2,130774599,16,7,9,2030-01-02 07:00:00,2030-01-02 20:00:00,POINT (-6.27217 53.36521)
522,arctic-charbonneau,307,2030-01-02 07:06:00,3,128942373,16,7,11,2030-01-02 07:00:00,2030-01-02 20:00:00,POINT (-6.27193 53.36967)
523,arctic-charbonneau,114,2030-01-02 07:08:00,4,1423546555,16,7,8,2030-01-02 07:00:00,2030-01-02 20:00:00,POINT (-6.26896 53.3738)
524,arctic-charbonneau,429,2030-01-02 07:10:00,5,308326739,16,7,14,2030-01-02 07:00:00,2030-01-02 20:00:00,POINT (-6.2699 53.37543)
...,...,...,...,...,...,...,...,...,...,...,...
272,unsorted-samson,64,2030-01-02 09:04:00,62,632666705,16,7,12,2030-01-02 07:00:00,2030-01-02 20:00:00,POINT (-6.27888 53.34987)
273,unsorted-samson,226,2030-01-02 09:06:00,63,5590594966,16,7,14,2030-01-02 07:00:00,2030-01-02 20:00:00,POINT (-6.27866 53.34963)
274,unsorted-samson,100,2030-01-02 09:08:00,64,1326700611,16,7,10,2030-01-02 07:00:00,2030-01-02 20:00:00,POINT (-6.28369 53.3471)
275,unsorted-samson,215,2030-01-02 09:10:00,65,5044959521,16,7,8,2030-01-02 07:00:00,2030-01-02 20:00:00,POINT (-6.28696 53.34323)


In [17]:
m.routes_

,route_name,truck_type,duration_min,distance_m,fuel_cost_€,labor_cost_€,truck_cost_€,total_cost_€,departure,arrival,utilization_time,utilization_load,utilization_rangelimit,geometry
0,arctic-charbonneau,2,62,23606,2.3606,39.06,184.8,226.2206,2030-01-02 07:01:00,2030-01-02 08:03:00,12.92,99.75,0.0,"LINESTRING (-6.28688 53.34197, -6.28684 53.354..."
1,champagne-perrault,2,59,17425,1.7425,37.17,184.8,223.7125,2030-01-02 07:01:00,2030-01-02 08:00:00,12.29,100.00,0.0,"LINESTRING (-6.28688 53.34197, -6.28876 53.357..."
2,cold-porcher,2,59,17104,1.7104,37.17,184.8,223.6804,2030-01-02 07:01:00,2030-01-02 08:00:00,12.29,99.75,0.0,"LINESTRING (-6.28688 53.34197, -6.27851 53.323..."
3,furious-dubois,2,58,28196,2.8196,36.54,184.8,224.1596,2030-01-02 07:01:00,2030-01-02 07:59:00,12.08,99.75,0.0,"LINESTRING (-6.28688 53.34197, -6.22693 53.357..."
4,impulsive-chaput,0,139,7153,28.6120,69.50,280.0,378.1120,2030-01-02 07:01:00,2030-01-02 09:20:00,28.96,99.60,0.0,"LINESTRING (-6.28688 53.34197, -6.27338 53.338..."
5,inclusive-samson,0,141,9139,36.5560,70.50,280.0,387.0560,2030-01-02 07:01:00,2030-01-02 09:22:00,29.38,98.40,0.0,"LINESTRING (-6.28688 53.34197, -6.28852 53.342..."
6,parallel-sartre,0,145,5586,22.3440,72.50,280.0,374.8440,2030-01-02 07:01:00,2030-01-02 09:26:00,30.21,98.90,0.0,"LINESTRING (-6.28688 53.34197, -6.28649 53.341..."
7,relative-legrand,0,141,9038,36.1520,70.50,280.0,386.6520,2030-01-02 07:01:00,2030-01-02 09:22:00,29.38,99.90,0.0,"LINESTRING (-6.28688 53.34197, -6.28072 53.340..."
8,silly-moreau,2,57,19309,1.9309,35.91,184.8,222.6409,2030-01-02 07:01:00,2030-01-02 07:58:00,11.88,97.50,0.0,"LINESTRING (-6.28688 53.34197, -6.26696 53.348..."
9,tan-lamar,2,59,14110,1.4110,37.17,184.8,223.3810,2030-01-02 07:01:00,2030-01-02 08:00:00,12.29,98.25,0.0,"LINESTRING (-6.28688 53.34197, -6.28688 53.340..."
